In [1]:
train_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/train_images"
val_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/val_images"
test_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/test_images/mistery_cat"

# AlexNet

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

# Directories
train_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/train_images"
val_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/val_images"
test_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/test_images/mistery_cat"

# Hyperparameters
batch_size = 32
epochs = 30
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and Dataloaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define AlexNet Model
model = models.alexnet(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(train_dataset.classes))  # Adjust output layer
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
    
    end_time = time.time()
    print(f"Training Time: {end_time - start_time:.2f} seconds")
    return model

# Evaluation Function
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

# Save the trained model
torch.save(trained_model.state_dict(), "alexnet_bird_classifier.pth")
print("Model saved as alexnet_bird_classifier.pth")

# Print model parameter count
param_count = sum(p.numel() for p in model.parameters())
print(f"Total Model Parameters: {param_count}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 221MB/s]


Epoch [1/30], Loss: 2.9994, Train Acc: 10.99%, Val Acc: 6.78%
Epoch [2/30], Loss: 3.0086, Train Acc: 4.91%, Val Acc: 4.66%
Epoch [3/30], Loss: 3.0106, Train Acc: 3.31%, Val Acc: 5.08%
Epoch [4/30], Loss: 3.0020, Train Acc: 3.84%, Val Acc: 5.08%
Epoch [5/30], Loss: 2.9991, Train Acc: 4.48%, Val Acc: 5.08%
Epoch [6/30], Loss: 2.9989, Train Acc: 4.27%, Val Acc: 5.08%
Epoch [7/30], Loss: 2.9983, Train Acc: 5.12%, Val Acc: 5.08%
Epoch [8/30], Loss: 2.9983, Train Acc: 4.16%, Val Acc: 5.08%
Epoch [9/30], Loss: 2.9979, Train Acc: 5.23%, Val Acc: 5.08%
Epoch [10/30], Loss: 2.9965, Train Acc: 4.06%, Val Acc: 5.08%
Epoch [11/30], Loss: 2.9969, Train Acc: 3.20%, Val Acc: 5.08%
Epoch [12/30], Loss: 2.9969, Train Acc: 5.12%, Val Acc: 5.08%
Epoch [13/30], Loss: 2.9969, Train Acc: 4.38%, Val Acc: 5.08%
Epoch [14/30], Loss: 2.9969, Train Acc: 4.80%, Val Acc: 5.08%
Epoch [15/30], Loss: 2.9966, Train Acc: 3.84%, Val Acc: 5.08%
Epoch [16/30], Loss: 2.9963, Train Acc: 3.84%, Val Acc: 5.08%
Epoch [17/30], L

# Resnet

In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

# Directories
train_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/train_images"
val_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/val_images"
test_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/test_images/mistery_cat"

# Hyperparameters
batch_size = 32
epochs = 30
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and Dataloaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define ResNet Model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.classes))  # Adjust output layer
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
    
    end_time = time.time()
    print(f"Training Time: {end_time - start_time:.2f} seconds")
    return model

# Evaluation Function
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

# Save the trained model
torch.save(trained_model.state_dict(), "resnet_bird_classifier.pth")
print("Model saved as resnet_bird_classifier.pth")

# Print model parameter count
param_count = sum(p.numel() for p in model.parameters())
print(f"Total Model Parameters: {param_count}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


Epoch [1/30], Loss: 2.0095, Train Acc: 42.37%, Val Acc: 22.46%
Epoch [2/30], Loss: 1.3168, Train Acc: 58.16%, Val Acc: 44.92%
Epoch [3/30], Loss: 1.0060, Train Acc: 67.56%, Val Acc: 53.81%
Epoch [4/30], Loss: 0.7765, Train Acc: 76.09%, Val Acc: 60.17%
Epoch [5/30], Loss: 0.6719, Train Acc: 80.58%, Val Acc: 56.36%
Epoch [6/30], Loss: 0.5522, Train Acc: 84.95%, Val Acc: 63.14%
Epoch [7/30], Loss: 0.5010, Train Acc: 84.20%, Val Acc: 69.49%
Epoch [8/30], Loss: 0.3390, Train Acc: 89.43%, Val Acc: 63.56%
Epoch [9/30], Loss: 0.3670, Train Acc: 87.51%, Val Acc: 55.08%
Epoch [10/30], Loss: 0.4148, Train Acc: 87.73%, Val Acc: 63.14%
Epoch [11/30], Loss: 0.2790, Train Acc: 91.78%, Val Acc: 60.17%
Epoch [12/30], Loss: 0.2434, Train Acc: 92.00%, Val Acc: 45.34%
Epoch [13/30], Loss: 0.2590, Train Acc: 91.68%, Val Acc: 62.71%
Epoch [14/30], Loss: 0.1945, Train Acc: 94.45%, Val Acc: 66.95%
Epoch [15/30], Loss: 0.2264, Train Acc: 93.28%, Val Acc: 65.68%
Epoch [16/30], Loss: 0.1912, Train Acc: 95.41%, V

# VGG

In [4]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

# Directories
train_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/train_images"
val_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/val_images"
test_dir = "/kaggle/input/croped-processed-augmented-bird-dataset/CropImage_Dataset/CropImage_Dataset/test_images/mistery_cat"

# Hyperparameters
batch_size = 32
epochs = 30
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and Dataloaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define VGG Model
model = models.vgg16(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(train_dataset.classes))  # Adjust output layer
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
    
    end_time = time.time()
    print(f"Training Time: {end_time - start_time:.2f} seconds")
    return model

# Evaluation Function
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

# Save the trained model
torch.save(trained_model.state_dict(), "vgg_bird_classifier.pth")
print("Model saved as vgg_bird_classifier.pth")

# Print model parameter count
param_count = sum(p.numel() for p in model.parameters())
print(f"Total Model Parameters: {param_count}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 183MB/s]


Epoch [1/30], Loss: 3.0965, Train Acc: 4.48%, Val Acc: 5.08%
Epoch [2/30], Loss: 3.0193, Train Acc: 4.91%, Val Acc: 7.63%
Epoch [3/30], Loss: 3.0220, Train Acc: 4.27%, Val Acc: 4.66%
Epoch [4/30], Loss: 3.0181, Train Acc: 3.74%, Val Acc: 4.66%
Epoch [5/30], Loss: 2.9930, Train Acc: 6.40%, Val Acc: 6.78%
Epoch [6/30], Loss: 3.0199, Train Acc: 5.76%, Val Acc: 5.08%
Epoch [7/30], Loss: 3.0181, Train Acc: 3.84%, Val Acc: 5.08%
Epoch [8/30], Loss: 3.0044, Train Acc: 3.95%, Val Acc: 5.08%
Epoch [9/30], Loss: 3.0055, Train Acc: 3.95%, Val Acc: 5.08%
Epoch [10/30], Loss: 3.0068, Train Acc: 4.06%, Val Acc: 5.08%
Epoch [11/30], Loss: 3.0060, Train Acc: 3.52%, Val Acc: 5.08%
Epoch [12/30], Loss: 3.0059, Train Acc: 4.70%, Val Acc: 5.08%
Epoch [13/30], Loss: 3.0065, Train Acc: 4.27%, Val Acc: 5.08%
Epoch [14/30], Loss: 3.0058, Train Acc: 3.84%, Val Acc: 4.66%
Epoch [15/30], Loss: 3.0020, Train Acc: 4.38%, Val Acc: 5.08%
Epoch [16/30], Loss: 3.0028, Train Acc: 3.63%, Val Acc: 5.08%
Epoch [17/30], Lo